In [1]:
%cd /content/
!pip install -q xlrd
!git clone https://a80b7012a5c512ec3c6cb546626348b891f4f77f@github.com/bwallace/evidence-inference --branch "span-location"

/content
fatal: destination path 'evidence-inference' already exists and is not an empty directory.


In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [3]:
%cd /content/evidence-inference
!git config --global user.email "lehman.e@husky.neu.edu"
!git pull origin span-location
!pip install gensim
!pip install spacy
!python -m spacy download en


/content/evidence-inference
From https://github.com/bwallace/evidence-inference
 * branch            span-location -> FETCH_HEAD
Already up to date.

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [4]:
%cd /content/evidence-inference/
import torch
import argparse
import os
import sys
import random

from os.path import join, dirname, abspath
mp1 = os.path.abspath(os.path.join('.//evidence_inference/preprocess'))
mp2 = os.path.abspath(os.path.join('.//evidence_inference/models/'))


module_paths = [mp1, mp2]
for mp in module_paths:
    if mp not in sys.path:
        sys.path.append(mp)
    
import preprocessor
from scan_regression import train_scan
USE_CUDA = False

/content/evidence-inference


In [0]:
train_Xy, inference_vectorizer = preprocessor.get_train_Xy(set(list(preprocessor.train_document_ids())[:1]), sections_of_interest=None, vocabulary_file=None, include_sentence_span_splits = True)

    


In [0]:
# create an internal validation set from the training data; use 90% for training and 10% for validation.
random.shuffle(train_Xy)
split_index = int(len(train_Xy) * .9)
val_Xy = train_Xy[split_index:]
train_Xy = train_Xy[:split_index]


In [0]:
test_Xy = preprocessor.get_Xy(set(list(preprocessor.validation_document_ids())[:1]), inference_vectorizer, sections_of_interest=None, include_sentence_span_splits = True) 



In [0]:
USE_CUDA = False
# train with 50 epochs, batch_size of 1, and patience of 3 (early stopping)
model = train_scan(inference_vectorizer, train_Xy, val_Xy, test_Xy, 100, 32, 5)


torch.save(model.state_dict(), './model_lr.pth')